In [1]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from Utility.Get_dataset_clean_function import download_data_from_blob


In [21]:
df = pd.read_csv('../Documentation/data_for_model_2018_dep_1.csv', sep=',')

In [22]:
df.drop(['Unnamed: 0'], axis=1, inplace=True)
df = df.drop(df[df['valeur_fonciere']<100000].index)

In [23]:
# Recuperation données pour construction modele ml

classe_liste_prefixe_voie = download_data_from_blob('ABREVIATION_VOIE.csv')
classe_liste_prefixe_voie = list(classe_liste_prefixe_voie['Abreviation voie retenue'].to_dict().values())
classe_liste_prefixe_voie = [i for i in classe_liste_prefixe_voie if i in df.columns.values]

classe_liste_code_culture = download_data_from_blob('CODE_CULTURE.csv')
classe_liste_code_culture = list(classe_liste_code_culture['Code_nature_culture'].to_dict().values())

classe_liste_code_culture_spe = download_data_from_blob('CODE_CULTURE_SPECIALE.csv')
classe_liste_code_culture_spe = list(classe_liste_code_culture_spe['CODE_CULTURE_SPECIALE'].to_dict().values())

# Multi PCA

In [24]:
pca_code_culture = PCA(n_components=10)
df_code_culture = df.loc[:, classe_liste_code_culture]
df.drop(classe_liste_code_culture, axis=1, inplace=True)

columns_name_code_culture = ['code_culture_' + str(i) for i in range(10)]

code_culture_reduct = pca_code_culture.fit_transform(df_code_culture)
code_culture_reduct_df = pd.DataFrame(data=code_culture_reduct, columns=columns_name_code_culture)

In [25]:
pca_prefixe_voie = PCA(n_components=10)
df_prefixe_voie = df.loc[:, classe_liste_prefixe_voie]
df.drop(classe_liste_prefixe_voie, axis=1, inplace=True)

columns_name_prefixe = ['code_prefixe_' + str(i) for i in range(10)]

prefixe_reduct = pca_prefixe_voie.fit_transform(df_prefixe_voie)
prefixe_reduct_df = pd.DataFrame(data=prefixe_reduct, columns=columns_name_prefixe)

In [26]:
pca_code_culture_spe = PCA(n_components=10)
classe_liste_code_culture_spe = [i for i in classe_liste_code_culture_spe if i in df.columns.values]
df_code_culture_spe = df.loc[:, classe_liste_code_culture_spe]
df.drop(classe_liste_code_culture_spe, axis=1, inplace=True)

columns_name_code_culture_spe = ['code_culture_spe_' + str(i) for i in range(10)]

code_culture_spe_reduct = pca_code_culture_spe.fit_transform(df_code_culture_spe)
code_culture_spe_reduct_df = pd.DataFrame(data=code_culture_spe_reduct, columns=columns_name_code_culture_spe)

In [27]:
df = pd.concat([df, prefixe_reduct_df, code_culture_reduct_df, code_culture_spe_reduct_df], axis=1)

In [28]:
X = df.drop(labels='valeur_fonciere', axis=1)
Y = df['valeur_fonciere']

In [29]:
del df, prefixe_reduct_df, code_culture_reduct_df, code_culture_spe_reduct_df, pca_code_culture_spe,pca_prefixe_voie,pca_code_culture,code_culture_spe_reduct,prefixe_reduct, code_culture_reduct

In [30]:
Y_100 = (Y / 100).__round__()
Y_10000 = (Y_100 / 100).__round__()

In [32]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y_100, train_size=0.7)

In [33]:
print(X_train.shape)
print(X_test.shape)

(8809, 53)
(3776, 53)


In [34]:
LG = LogisticRegression()
LG.fit(X_train,Y_train)

ValueError: Input X contains NaN.
LogisticRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

# Random Forest

In [29]:
model = RandomForestClassifier(max_depth=50)

In [31]:
model.fit(X_train,Y_train)

MemoryError: could not allocate 358875136 bytes

In [ ]:
prediction = model.predict(X_test)

In [ ]:
prediction

In [ ]:
c = 0
for x, y in zip(Y_test, prediction):
    if x == y:
        c += 1

BOW_ACCURACY = c / Y_test.shape[0]

print('Accuracy:' + str(BOW_ACCURACY))